In [1]:
import pandas as pd
import numpy as np
import sqlite3
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
EXPERIMENT_NAME = 'P3856'
RUN_NAME = 'P3856_YHE211_1_Slot1-1_1_5104'

In [3]:
MZ_TOLERANCE_PPM = 5
MZ_TOLERANCE_PERCENT = MZ_TOLERANCE_PPM * 10**-4
SCAN_TOLERANCE = 2
RT_TOLERANCE = 1

##### 3DID parameters

##### PASEF parameters

In [4]:
# load the detected features
EXPERIMENT_DIR = '/data2/experiments/{}'.format(EXPERIMENT_NAME)
FEATURES_DIR = '{}/recalibrated-features'.format(EXPERIMENT_DIR)
FEATURES_FILE = '{}/experiment-features.pkl'.format(FEATURES_DIR)
FEATURES_DEDUP_FILE = '{}/exp-{}-run-{}-features-pasef-dedup.pkl'.format(FEATURES_DIR, EXPERIMENT_NAME, RUN_NAME)
MZ_COLUMN_NAME = 'recalibrated_monoisotopic_mz'

In [5]:
RT_LOWER = 1650
RT_UPPER = 2200

features_df = pd.read_pickle(FEATURES_FILE)
# features_df = features_df[(features_df.run_name == RUN_NAME) & (features_df.rt_apex >= RT_LOWER) & (features_df.rt_apex <= RT_UPPER)]
features_df = features_df[(features_df.rt_apex >= RT_LOWER) & (features_df.rt_apex <= RT_UPPER)]

In [6]:
print('there are {} features prior to de-dup'.format(len(features_df)))

there are 168681 features prior to de-dup


In [7]:
# features_df['mz'] = features_df.recalibrated_monoisotopic_mz  # shorthand to reduce verbosity
features_df['mz'] = features_df[MZ_COLUMN_NAME]  # shorthand to reduce verbosity
features_df['mz_ppm_tolerance'] = features_df.mz * MZ_TOLERANCE_PERCENT / 100
features_df['mz_lower'] = features_df.mz - features_df.mz_ppm_tolerance
features_df['mz_upper'] = features_df.mz + features_df.mz_ppm_tolerance
features_df['scan_lower'] = features_df.scan_apex - SCAN_TOLERANCE
features_df['scan_upper'] = features_df.scan_apex + SCAN_TOLERANCE
features_df['rt_lower'] = features_df.rt_apex - RT_TOLERANCE
features_df['rt_upper'] = features_df.rt_apex + RT_TOLERANCE
# features_df['composite_key'] = features_df.apply(lambda row: '{},{},{}'.format(row.feature_id, row.precursor_id, row.percolator_idx), axis=1)
features_df['composite_key'] = features_df.apply(lambda row: '{},{}'.format(row.feature_id, row.precursor_id), axis=1)

In [8]:
# remove these after we're finished
columns_to_drop_l = []
columns_to_drop_l.append('mz')
columns_to_drop_l.append('mz_ppm_tolerance')
columns_to_drop_l.append('mz_lower')
columns_to_drop_l.append('mz_upper')
columns_to_drop_l.append('scan_lower')
columns_to_drop_l.append('scan_upper')
columns_to_drop_l.append('rt_lower')
columns_to_drop_l.append('rt_upper')
columns_to_drop_l.append('composite_key')


In [9]:
features_df.sort_values(by=['intensity'], ascending=False, inplace=True)

In [10]:
# see if any detections have a duplicate - if so, find the dup with the highest intensity and keep it
keep_l = []
for row in features_df.itertuples():
    dup_df = features_df[(features_df.mz > row.mz_lower) & (features_df.mz < row.mz_upper) & (features_df.scan_apex > row.scan_lower) & (features_df.scan_apex < row.scan_upper) & (features_df.rt_apex > row.rt_lower) & (features_df.rt_apex < row.rt_upper)].copy()
    # group the dups by charge - take the most intense for each charge
    for group_name,group_df in dup_df.groupby(['charge'], as_index=False):
        keep_l.append(group_df.iloc[0].composite_key)


In [11]:
# remove any features that are not in the keep list
dedup_df = features_df[features_df.composite_key.isin(keep_l)].copy()

In [12]:
number_of_dups = len(features_df)-len(dedup_df)
print('removed {} duplicates ({}% of the original detections)'.format(number_of_dups, round(number_of_dups/len(features_df)*100)))
print('there are {} detected de-duplicated features'.format(len(dedup_df)))

removed 15307 duplicates (9% of the original detections)
there are 153374 detected de-duplicated features


In [13]:
# remove the columns we added earlier
dedup_df.drop(columns_to_drop_l, axis=1, inplace=True)

In [14]:
dedup_df.sample(n=3)

,feature_id,charge,rt_apex,rt_peak_width,scan_apex,scan_peak_width,intensity,intensity_full_rt_extent,precursor_id,monoisotopic_mass,predicted_mass_error,recalibrated_monoisotopic_mass,recalibrated_monoisotopic_mz,percolator_idx,run_name
4,3902905,1,2123.85,50.01,161.51,66.22,2055.0,1134,39029,1055.782555,0.006247,1055.776308,1056.783584,28,P3856_YHE211_1_Slot1-1_1_5104
5,1667606,1,1779.59,20.00,345.88,75.00,306.0,908,16676,982.567766,0.009386,982.558381,983.565657,28,P3856_YHE211_1_Slot1-1_1_5104
3,2310104,1,1883.75,20.21,429.65,89.25,662.0,1335,23101,1192.728333,0.003948,1192.724385,1193.731661,28,P3856_YHE211_1_Slot1-1_1_5104


In [15]:
dedup_df.to_pickle(FEATURES_DEDUP_FILE)